In [1]:
from mofa_empirical import *
import geemap, ee
from mpl_toolkits import mplot3d
from datetime import datetime
import ipywidgets as widgets
from ipyleaflet import WidgetControl, Marker
from os import path

In [2]:
# ****** widgets of interactive system
#year_range, crop_mask, data_type

submit = widgets.Button(description='Submit', button_style='primary', tooltip='Click me')
clear = widgets.Button(description='Clear')

style = {'description_width': 'initial'}

region_choice = widgets.Dropdown(
    description='Region',
    options=[
        'Northern Region',
        'Greater Accra Region',
        'Northern & Greater Accra Regions',
        'Ghana (whole country)'
    ],
    style = style,
    disabled = False,
    value='Northern Region'
)

year_range = widgets.Dropdown(
    description='Year',
    options=[
        str(x) for x in range(2006, 2019)
    ],
    style = style,
    value='2006'
)

crop_mask = widgets.Dropdown(
    description='Maize Mask',
    options=[
        'MODIS Land Cover Type 1 (IGBP) Grasslands: dominated by herbaceous annuals (<2m)',
        'MODIS Land Cover Type 2 (UMD) Grasslands: dominated by herbaceous annuals (<2m)',
        'MODIS Land Cover Type 3 (LAI) Grassland: dominated by herbaceous annuals (<2m) including cereal croplands',
        'MODIS Land Cover Type 2 (UMD) Cropland: at least 60% of area is cultivated cropland',
        'CAU Maize classification'
    ],
    style = style,
    disabled = False,
    value='MODIS Land Cover Type 2 (UMD) Grasslands: dominated by herbaceous annuals (<2m)'
)

data_source = widgets.Dropdown(
    description='Data source',
    options=[
        'Max LAI over growing season, between Jun and Nov',
        'ERA5 accummulated precipitation (mm) between Jun and Nov',
        'ERA5 mean temperature at 2m (deg K) between Jun and Nov',
        'CAU Maize Map',
        'MOFA yield statistics (1000 kg/ha)',        
        'Empirical Yield = MaxLAI*1500-700 (kg/ha)',
        'Maize Planting Area (ha)'
    ],
    style = style,
    value='Max LAI over growing season, between Jun and Nov'
)

# *** checkbox widgets ***
checkbox_mask = widgets.Checkbox(
    value = True, 
    description = "Apply maize mask",
    style = style)

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')


In [46]:
def create_maize_area_map_jose(aoi=ghana_district):
    # TO DO: can use "Crop_production_Data_SRID.csv" file instead
    df_crop_area = pd.read_csv('Qtrend/crop_area-Jose.csv', dtype={'name_adm2': str, 'maiz_r': np.float64})\
                                .sort_values('maiz_r', ascending=False)

    
    # set a new varible with value from df_maize TO CREATE A NEW MAP LAYER
    features = [] # list of ee.Features
    for i, dist in enumerate(aoi.aggregate_array('ADM2_NAME').getInfo()): 
        aoi_dist = aoi.filter(ee.Filter.eq('ADM2_NAME', dist))\
                                        .first().geometry()
        if dist in df_crop_area['name_adm2'].unique():
            
            maiz_a = df_crop_area[df_crop_area['name_adm2']==dist]['maiz_r']
            area = maiz_a.values[0] if len(maiz_a) >0 else -999
        else:
            #print(dist, 'has no planting area recorded.')
            area = -999
            
        new_feature = ee.Feature(aoi_dist)
        new_feature = new_feature.set({'id':i, 'maize_area':area, 'ADM2_NAME': dist})

        features.append( new_feature ) #aoi_dist.set('yield', yi) )
        
    maize_areas = ee.FeatureCollection(features)
    return maize_areas


In [62]:
def create_mofa_yield_map(year, withPlantingArea=False, region='northern'):
    '''region options: 'northern', 'greater accra', or 'all'  '''
    region = region.upper()
    
    df = pd.read_csv('Qtrend/Ghana_Distirct_level_Crop_Yield_Data-upto2020.csv', sep=',', header=0)
    df_maize = df[(df['CROP']=='MAIZE') & (df['YEAR']== year)]
    
    if region!='ALL':
        df_maize = df_maize[df_maize['REGION'] == region]
    
    df_maize = df_maize.replace('Tamale Metro','Tamale North Sub Metro')    
    df_maize = df_maize.replace('Central Gonja','Gonja Central')
    df_maize = df_maize.replace('East Mamprusi','Mamprusi East')
    df_maize = df_maize.replace('Tamale Metro','Tamale North Sub Metro')
    df_maize = df_maize.replace('Yendi','Yendi Municipal')
    #print(df_maize['DISTRICT'].unique())
    
    if withPlantingArea:
        ''' # use "Crop_production_Data_SRID.csv" file instead
        df_crop_area = pd.read_csv('Qtrend/crop_area-Jose.csv', \
                            dtype={'name_adm2': str, 'maiz_r': np.float64})\
                            .sort_values('maiz_r', ascending=False)'''

        df_crop_area = pd.read_csv('Qtrend/Crop_production_Data_SRID.csv', \
                            dtype={'DISTRICT': str, 'AREA (Ha)': np.float64, 'YEAR': int})\
                            .sort_values('AREA (Ha)', ascending=False)
        print(df_crop_area['CROP'].unique())
        df_crop_area = df_crop_area[(df_crop_area['CROP']=='Maize') & (df_crop_area['YEAR']==year)]
        
    # set a new varible with value from df_maize TO CREATE A NEW MAP LAYER
    yield_features = [] # list of ee.Features
    for i, dist in enumerate(df_maize['DISTRICT'].unique()): 
        if dist in ghana_district.aggregate_array('ADM2_NAME').getInfo():
            yi = df_maize[df_maize['DISTRICT']==dist]['YIELD'].values[0]

            aoi_dist = ghana_district.filter(ee.Filter.eq('ADM2_NAME', dist))\
                                        .first().geometry()
            
            new_feature = ee.Feature(aoi_dist)
            new_feature = new_feature.set({'id':i, 'yield': yi, 'ADM2_NAME': dist})
            
            if withPlantingArea:
                #maiz_a = df_crop_area[df_crop_area['name_adm2']==dist]['maiz_r']
                maiz_a = df_crop_area[df_crop_area['DISTRICT']==dist]['AREA (Ha)']
                area = maiz_a.values[0] if len(maiz_a) >0 else -999
                new_feature = new_feature.set({ 'maize_area':area})

            #print(dist, '*******', new_feature.get('yield').getInfo())
            
            ## TO DO: PRE-STORE this layer, add re use 
            
            yield_features.append( new_feature ) #aoi_dist.set('yield', yi) )
        #else: 
            #print(dist, 'not in GAUL names')
            
    mofa_yield = ee.FeatureCollection(yield_features)
    #print(mofa_yield.first().getInfo())
    
    return mofa_yield

In [44]:
# make MOFA YIELD VECTOR to be stored: two layers: yield and planting area
for year in range(2006, 2021):
    print(year)
    mofa_yield = create_mofa_yield_map(year, withPlantingArea=True, region='all')
    #image = mofa_yield.reduceToImage(properties=['yield'], reducer=ee.Reducer.first()).rename('yield')
    #geemap.ee_export_image(image, filename=f'MOFA_Yield_Image_Northern_{year}.tif', scale=500, region=northern_district, file_per_band=False)
    geemap.ee_to_shp(mofa_yield, f'Qtrend/shp/MOFA_Yield_and_Area_vector_{year}.shp')
    

2006
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/shp/MOFA_Yield_and_Area_vector_2006.shp
2007
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/shp/MOFA_Yield_and_Area_vector_2007.shp
2008
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/shp/MOFA_Yield_and_Area_vector_2008.shp
2009
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/shp/MOFA_Yield_and_Area_vector_2009.shp
2010
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/shp/MOFA_Yield_and_Area_vector_2010.shp
2011
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/shp/MOFA_Yield

In [50]:
# make MOFA YIELD RASTER to be stored: single layer without Planting area
for year in range(2015, 2021):
    print(year)
    mofa_yield = create_mofa_yield_map(year, region='all')
    image = mofa_yield.reduceToImage(properties=['yield'], reducer=ee.Reducer.first()).rename('yield')
    #Map.addLayer(image)
    geemap.ee_export_image(image, filename=f'Qtrend/MOFA_Yield_Image_{year}.tif', scale=500, \
                           region=ghana_country.geometry(), file_per_band=False)
    #geemap.ee_to_shp(mofa_yield, f'Qtrend/shp/MOFA_Yield_Image_{year}.shp')
#Map   

2015
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/MOFA_Yield_Image_2015.tif
2016
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/MOFA_Yield_Image_2016.tif
2017
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/MOFA_Yield_Image_2017.tif
2018
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/MOFA_Yield_Image_2018.tif
2019
Generating URL ...
An error occurred while downloading.
Invalid JSON payload received. Unexpected token.
: {"constantValue": NaN}}}}, "value": {"
                    ^
2020
Generating URL ...
Please wait ...
Data downloaded to /Users/qingling/Documents/DATA/Ghana-Crop/Workshop2022/notebooks/Qtrend/MOFA_Yield_Image_2020.tif


In [63]:
def create_maize_area_map(year, aoi=ghana_district):
    # DONE: use "Crop_production_Data_SRID.csv" file instead
    df_crop_area = pd.read_csv('Qtrend/Crop_production_Data_SRID.csv', \
                            dtype={'DISTRICT': str, 'AREA (Ha)': np.float64, 'YEAR': int})\
                            .sort_values('AREA (Ha)', ascending=False)
    df_crop_area = df_crop_area[(df_crop_area['CROP']=='Maize') & (df_crop_area['YEAR']==year)]
        
    # set a new varible with value from df_maize TO CREATE A NEW MAP LAYER
    features = [] # list of ee.Features
    for i, dist in enumerate(aoi.aggregate_array('ADM2_NAME').getInfo()): 
        aoi_dist = aoi.filter(ee.Filter.eq('ADM2_NAME', dist))\
                                        .first().geometry()
        if dist in df_crop_area['DISTRICT'].unique():
            
            maiz_a = df_crop_area[df_crop_area['DISTRICT']==dist]['AREA (Ha)']
            area = maiz_a.values[0] if len(maiz_a) >0 else -999
            #print(dist, 'planting area recorded.')
            print(maiz_a)
        else:
            #print(dist, 'has no planting area recorded.')
            area = -999
            
        new_feature = ee.Feature(aoi_dist)
        new_feature = new_feature.set({'id':i, 'maize_area':area, 'ADM2_NAME': dist})

        features.append( new_feature ) #aoi_dist.set('yield', yi) )
        
    maize_areas = ee.FeatureCollection(features)
    return maize_areas


In [64]:
# make MOFA AREA RASTER to be stored: single layer without Planting area
for year in range(2006, 2007):
    print(year)
    mofa_area = create_maize_area_map(year, aoi=ghana_district)
    image = mofa_area.reduceToImage(properties=['maize_area'], reducer=ee.Reducer.first()).rename('area')
    #Map.addLayer(image)
    geemap.ee_export_image(image, filename=f'Qtrend/MOFA_Area_Image_{year}.tif', scale=500, \
                           region=ghana_country.geometry(), file_per_band=False)   

2006
Generating URL ...
An error occurred while downloading.
Description length exceeds maximum.


In [9]:
def clear_clicked(button):
    Map.clear()
    Map.addLayer(outline2, {'palette': '999999'}, 'Ghana districts')
    Map.addLayer(outline, {'palette': '000000'}, 'Northern & Greater Accra districts')
    
    
def checkmask_clicked(button):
    if checkbox_mask.value == True:
        crop_mask.disabled = False
    else:
        crop_mask.disabled = True
        
    
def submit_clicked(button):
    
    '''Add choice of region to focus on'''
    if region_choice.index == 0:
        aoi = northern_district
        Map.center = tamale_centre
        Map.zoom = 8
        #Map.setCenter(lon, lat, zoom)
    elif region_choice.index == 1:
        aoi = greateraccra_district
        Map.center = accra_centre
        Map.zoom = 9
    elif region_choice.index == 2:
        aoi = northern_district.merge(greateraccra_district) 
        Map.center = zoom_centre
        Map.zoom = 7
    elif region_choice.index == 3:
        ghana_province = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))
        aoi = ghana_province
        Map.center = zoom_centre
        Map.zoom = 7
        #Map.remove_layer('Northern & Accra districts')
        
    region_name = list(set(aoi.aggregate_array('ADM1_NAME').getInfo()))
    #print(region_name)
    
    #Map.clear_colorbar()
    output_widget.clear_output()
    if len(Map.layers) > base_no_layers: #>3
        # layers[-1] should be the vector polygpons
        Map.remove_layer(Map.layers[-2])
    if len(Map.controls) > base_no_controls: #>8
        #  to remove pallete
        Map.remove_control(Map.controls[-1])
        
    year = int(year_range.value)
    iMask = crop_mask.index if checkbox_mask.value else -1
    iData = data_source.index
        
    if iData == 0: #'Max LAI over growing season': 
        band_yearly, band_reduced = load_modis_band('Lai', year, aoi, 'max')
        data = band_reduced.clip(aoi)
        vis = colorized_vis['Lai']
        title = f'Max LAI {year}'
        legend_label = 'Max LAI over growing season'
        
    elif iData == 1: #'ERA5 accummulated precipitation (mm)'
        era5_in_season = load_era_band('total_precipitation', year, aoi)
        data = era5_in_season.sum().clip(aoi).multiply(100)
        vis = colorized_vis['Prec']
        title = f'Precipitation {year}'
        legend_label = 'Accummulated precipitation (mm)'
        
    elif iData == 2: #'ERA5 temperature at 2m'
        era5_in_season = load_era_band('temperature_2m', year, aoi)
        accum_temp = era5_in_season.sum().clip(aoi)
        data = accum_temp.divide(6) #.subtract(273.15) # six monthly temp    
        vis = {
                'min': 295.0,
                'max': 302.0,
                'palette': [
                "#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
                "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
                "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF"] 
                }
        title = f'Temperaure {year}'
        legend_label = 'Mean temperature at 2m, Degree K'
        
    elif iData == 3: #'CAU Maize Map'
        maize_imgCol = ee.ImageCollection('users/xianda19/classification_result/2021/Ghana/maize_20210501_20211011_100percentSamples')
        data = maize_imgCol.mosaic().selfMask() 
        vis = {
                'min':0, 
                'max':1
                }
        title = "CAU maize layer"
        
        # ---------use the next two lines if to erode S2 to MODIS resolution:-------
        #kernel = ee.Kernel.square(1, 'meters')
        #data = data.focalMin(kernel = kernel, iterations=25)
        #---------------------------------------------------------------------------
        
    elif iData == 4: #'MOFA yield statistics'
        for i, name in enumerate(region_name):
            print('Processing', name, '...')
            if i == 0:
                mofa_yield = create_mofa_yield_map(year, region=name)
            else:
                mofa_yield = mofa_yield.merge(create_mofa_yield_map(year, region=name))
        data = mofa_yield.reduceToImage(properties=['yield'], reducer=ee.Reducer.first()).rename('yield')
            
        vis = {
                'min': 0.5,
                'max': 3.5,
                'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
                }
        title = f'MOFA Yield {year}'
        legend_label = 'MOFA Yield, 1000 kg/ha'
                  
    elif iData == 5: #'Empirical Yield = MaxLAI*1500-700 (kg/ha)'
        band_yearly, band_reduced = load_modis_band('Lai', year, aoi, 'max')
        maxlai_img = band_reduced.clip(aoi)
        data = maxlai_img.multiply(1.5).subtract(0.7).rename('yield_empirical')
        vis = {
                'min': 0.5,
                'max': 5.5,
                'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
                }
        title = f'Empirical Yield calculated from MaxLAI {year}'
        legend_label = 'Empirical Yield, 1000 kg/ha'
        
    elif iData == 6: # Planted area of maize 
        areas = create_maize_area_map(aoi=aoi)
        data = areas.reduceToImage(properties=['maize_area'], reducer=ee.Reducer.first()).rename('area')
        vis = {
                'min': 6000,
                'max': 16000,
                'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
                }
        title = f'Maize Planting area'
        legend_label = 'maize area, ha' # TO-DO: double check if this is ha?
    
    if checkbox_mask.value:
        if iMask == 0: #LC_Type1 Grassland
            lc_crop, lc_all = load_modis_lc('LC_Type1', year, aoi, cropclasses=[10])
        elif iMask == 1: #LC_Type2 Grassland
            lc_crop, lc_all = load_modis_lc('LC_Type2', year, aoi, cropclasses=[10])
        elif iMask == 2: #LC_Type3 Grassland
            lc_crop, lc_all = load_modis_lc('LC_Type3', year, aoi, cropclasses=[1])
        elif iMask == 3: #LC_Type2 Cropland
            lc_crop, lc_all = load_modis_lc('LC_Type2', year, aoi, cropclasses=[12])
        elif iMask == 4: # CAU
            #kernel = ee.Kernel.square(10, units='meters') 
            maize_imgCol = ee.ImageCollection('users/xianda19/classification_result/2021/Ghana/maize_20210501_20211011_100percentSamples')
            lc_crop = maize_imgCol.mosaic().selfMask()           
            #lc_crop = lc_crop.focalMin(kernel = kernel, iterations=25)
            
        data = data.mask(lc_crop)
    
    Map.addLayer(data, vis, title)
    
    if iData != 3:
        Map.add_colorbar(vis, label=legend_label, layer_name=title,
                        orientation="vertical", transparent_bg=True)

    
    #previous_iData = iData
    # test prints
    #with output_widget: print(year, iMask, iData)
    if region_choice.index == 3:
        Map.addLayer(outline2, {'palette': '999999'}, 'Ghana districts')
    else:
        Map.addLayer(outline, {'palette': '000000'}, 'Northern & Greater Accra districts')
    #Map.addLayer(outline1, {'palette': '000000'}, 'Greater Accra districts')
        

In [10]:
def handle_click(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        
        with output_widget:
            output_widget.clear_output()
            if len(Map.controls) > base_no_controls:
                Map.remove_control(Map.controls[-1])
                
            print(f'{latlon[0]:0.5f}', f'{latlon[1]:0.5f}')
            poi = ee.Geometry.Point(latlon[::-1])
            
            province = ghana_district.filterBounds(poi).aggregate_array('ADM1_NAME').getInfo()
            district = cities.filterBounds(poi).aggregate_array('ADM2_NAME').getInfo()
            country = cities.filterBounds(poi).aggregate_array('ADM0_NAME').getInfo()
            
            if len(province) > 0:
                # poi within ghana
                print(f"{district[0]}, {province[0]}")
            else:
                print(f"{district[0]}, {country[0]}")
            
    Map.default_style = {'cursor': 'pointer'}

In [11]:
accra_centre = [5.8, 0.172]
zoom_centre = [8, -1.33]

In [8]:
# ###### define the study area and corresponding maize mask ######
cities = ee.FeatureCollection("FAO/GAUL/2015/level2")
ghana_district = cities.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))
northern_district = ghana_district.filter(ee.Filter.eq('ADM1_NAME', 'Northern'))
greateraccra_district = ghana_district.filter(ee.Filter.eq('ADM1_NAME', 'Greater Accra'))

outline = ee.Image().paint(northern_district.merge(greateraccra_district), 0, 1)
outline2 = ee.Image().paint(ghana_district, 0.2, 0.5)

Map = geemap.Map(center=tamale_centre, zoom=8) 
Map.addLayer(outline2, {'palette': '999999'}, 'Ghana districts')
Map.addLayer(outline, {'palette': '000000'}, 'Northern & Accra districts')

#Map.add_control(output_control)
full_widgets = widgets.VBox([
  widgets.HBox([Map]),
  widgets.HBox([year_range, crop_mask, data_source], layout=widgets.Layout(height="40px")),
  widgets.HBox([region_choice, checkbox_mask, submit, clear], layout=widgets.Layout(height="40px"))
])

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomleft')
Map.add_control(output_control)


base_no_layers = len(Map.layers)
base_no_controls = len(Map.controls)

Map.on_interaction(handle_click)
submit.on_click(submit_clicked)
clear.on_click(clear_clicked)
checkbox_mask.observe(checkmask_clicked)


display(full_widgets)


# Interactive Map for inspecting time series

In [9]:
maize_area = create_maize_area_map(aoi=northern_district.merge(greateraccra_district))
maize_area_data = maize_area.reduceToImage(properties=['maize_area'], reducer=ee.Reducer.first()).rename('area')
vis = {
        'min': 6000,
        'max': 16000,
        'palette': ['FCFDBF', 'FDAE78', 'EE605E', 'B63679', '711F81', '2C105C']
        }

In [15]:
Map1 = geemap.Map(center=farm_7021YAM, zoom=8)
Map1.addLayer(outline, {}, 'Districts')

# Add an output widget to the map
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomleft')
Map1.add_control(output_control)

Map1.addLayer(maize_area_data, vis, 'Maize Planting Area', opacity=0.7)
Map1.add_colorbar(vis, label='maize area, ha', layer_name='Maize Planting Area',
                        orientation="horizontal", transparent_bg=True)

dummy_file = open("Qtrend/Qtrend_ALL_NORTHERN_DISTRICTS.png", "rb")
image = dummy_file.read()
dummy1 = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)
image_widget = WidgetControl(widget=dummy1, position='topright')
Map1.add_control(image_widget)

def refresh_trend_map():
    Map1.clear_controls()
    Map1.add_colorbar(vis, label='maize area, ha', layer_name='Maize Planting Area',
                        orientation="horizontal", transparent_bg=True)
    Map1.add_control(output_control)
    Map1.add_control(image_widget)

#orignal_contrs = Map1.controls

# Capture user interaction with the map
def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map1.default_style = {'cursor': 'wait'}
        
        with output_widget:
            output_widget.clear_output()
            if len(Map1.controls) >= 11:
                Map1.remove_control(Map1.controls[-1])
                
            print(f'{latlon[0]:0.5f}', f'{latlon[1]:0.5f}')
            poi = ee.Geometry.Point(latlon[::-1])
            #point = ee.Image().paint(poi, 0, 10)
            #Map1.addLayer(point)
                        
            district = northern_district.merge(greateraccra_district).filterBounds(poi).aggregate_array('ADM2_NAME').getInfo()
            if len(district) > 0:
                district = district[0]
                print(district)
                
                #polygon = ee.Image().paint(**{'featureCollection': aoi_dist })
                #polygonIntersects = polygon.geometry().intersects(poi)
                #print(polygonIntersects.getInfo())
            
                short_name = ''.join(district.split(' ')).replace('/', '_')
                
                img_fn = f"Qtrend/Qtrend_{short_name}.png"
                if path.exists(img_fn):
                    file = open(img_fn, "rb")
                else:
                    file =  open("Qtrend/Qtrend_ALL_NORTHERN_DISTRICTS.png", "rb")
                image = file.read()
                img_widget = widgets.Image(
                    value=image,
                    format='png',
                    width=300,
                    height=400,
                )
                image_widget = WidgetControl(widget=img_widget, position='topright')
                Map1.add_control(image_widget)
                
                img_fn_qc = f"Qtrend/Qtrend_{short_name}_ModisHist_2015.png"
                if path.exists(img_fn_qc):
                    file = open(img_fn_qc, "rb")
                else:
                    file =  open("Qtrend/Qtrend_ALL_NORTHERN_DISTRICTS.png", "rb")
                image = file.read()
                img_widget = widgets.Image(
                    value=image,
                    format='png',
                    width=300,
                    height=400,
                )
                image_widget = WidgetControl(widget=img_widget, position='topright')
                Map1.add_control(image_widget)
            else:
                district = cities.filterBounds(poi).aggregate_array('ADM2_NAME').getInfo()
                country = cities.filterBounds(poi).aggregate_array('ADM0_NAME').getInfo()
                print(f"{district[0]}, {country[0]}")
            
    Map1.default_style = {'cursor': 'pointer'}

Map1.on_interaction(handle_interaction)
Map1

Map(center=[9.70065, -0.54129], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

# Friday morning to report back

0) ***legend/palette issue***  
1) all districts, instead of northern  
2) some demo of toolboxes : e.g. upload own vector / raster  
3) user-defined AOI  
4) drawing tools: TBD: remove?  
5) ***apply-mask as default***  
6) on-click: per-pixel TS explorer   
     TS-  
7) graph of ALL properties of pixel, on click     
8) yield estimate to be related: e.g average yield over the masked area  
 
1) ***axis labels (year, lai, precp, && their units) for TS figures***  
2) •••upload mofa csv file to GitHub  
3) ***add scatterplots as subplots: mofa yield vs precp; mofa yield vs empirical yield; and show stats of graph***  
4) displaying district/county names on click

# 11-Mar-2022 notes: generate fig (scatter plots) to illustrate MODIS LAI not useful for yield prediction:
1) generate histogram per QA level,  
2) generate IO webpage,  
3) Tidy up, fix all bugs,   
4) change rain m to mm,  
5) add 1-to-1 line for scatterplot

In [ ]:
BEIS REPORT NO. 4 
researchfish 

In [ ]:
https://data.gov.gh/group/agriculture-sector 